In [ ]:
#Necessary Imports
!pip install yfinance -q
import json, requests, datetime, random, matplotlib.pyplot as plt, csv, yfinance as yf, pandas as pd

#Rates for calculation
risk_free = .0457
expected_rate = .06
reces = .25
normal = .60
boom = .15

#Time Value of Money function
def calculate_future_value(N, IY, PV, PMT):
    """
    Calculates the Future Value (FV) using TVM principles.

    Parameters:
        N (int): Number of periods.
        IY (float): Interest rate per period (as a percentage, not decimal).
        PV (float): Present value (negative for cash outflows).
        PMT (float): Payment per period (negative for cash outflows).

    Returns:
        float: The Future Value (FV).
    """
    # Convert interest rate from percentage to decimal
    rate = IY / 100

    # Calculate Future Value
    FV = PV * (1 + rate) ** N + PMT * (((1 + rate) ** N - 1) / rate)
    return FV

#Machine Learning functions: Train and test data
def predict(train, test, predictors, model):
  model.fit(train[predictors], train['Target'])
  preds = model.predict_proba(test[predictors])[:,1]
  preds[preds >=.6] = 1
  preds[preds <.6] = 0
  preds = pd.Series(preds, index = test.index, name = "Predictions")
  combined = pd.concat([test["Target"], preds], axis = 1)
  return combined

def backtest(data, model, predictors, start = 2500, step = 250):
  all_predictions = []

  for i in range(start, data.shape[0], step):
    train = data.iloc[0:i].copy()
    test = data.iloc[i:(i + step)].copy()
    predictions = predict(train, test, predictors,model)
    all_predictions.append(predictions)
    return pd.concat(all_predictions)

if __name__ == "__main__":

  #option 1 list data
  close_list = []
  date_list = []
  int_list = []
  number_list = []
  value_over_time_list = []
  years_list = []

  #option 2 list data
  random_num = []
  name_list = []
  symbol_list = []
  selected_stocks = []
  expacted_return_list = []
  profolio_expected_list = []
  profolio_value_over_time_list = []
  years_profolio_list = []

  print("\nDISCLAIMER: This application is for educational purposes and should not be used for financial advice\n")
  print("Welcome to stock profolio real time generator!")
  print("\nOptions:\n\nIndividual Stock Information = 1,\n\nGenerate a random profolio for me = 2,\n\nSee if a stock will go up or down using machine learning = 3")

  #Repeat if user wants to continue
  repeat = True
  try:
    while repeat == True:
      user_input = input("What Would you like to do today?")

      #Option 1 for user
      if user_input == "1":

        #Asking user for needed information for TMV
        stock = input("What stock would you like to see more infomation about? (Stock Symbol) ")
        pv = float(input("How much would you like to invest? "))
        n = float(input("How many years would you like to invest for? "))
        pmt = float(input("How much would you like to invest yearly? "))
        pv = int(pv)
        n = int(n)
        pmt = int(pmt)

        #Requesting needed APIs'
        stock_info = requests.get(f"https://api.twelvedata.com/time_series?symbol={stock},EUR/USD,ETH/BTC:Huobi,TRP:TSX&interval=1day&apikey=782c1e9d7e864db8999cf5607d1cf9c8")
        aplha_vantage_balancesheet = requests.get(f"https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol={stock}&apikey=YY9CDSZHMTJFS6JV")
        alpha_vantage_income_statement = requests.get(f"https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={stock}&apikey=YY9CDSZHMTJFS6JV")
        beta_info = requests.get(f"https://api.newtonanalytics.com/stock-beta/?ticker={stock}&index=^GSPC&interval=1mo​&observations=100")

        #Getting stock close price and date information
        if stock_info:
          #Error Handling if user did not input correct stock symbol
          try:
            stock_data = json.loads(stock_info.text)
            stock_price_data = stock_data[f"{stock}"]["values"]
          except KeyError:
            print("Invalid Stock Symbol")
            continue

          #Drilling down to get datetime and close price
          for data in stock_price_data:
            date_time = data["datetime"]
            close_price = data["close"]
            close_list.append(close_price)
            date_list.append(date_time)

        #Getting stock balancesheet information
        if aplha_vantage_balancesheet:
          bs_data = json.loads(aplha_vantage_balancesheet.text)
          common_stock = bs_data["annualReports"][0]["commonStockSharesOutstanding"]
          equity = bs_data["annualReports"][0]["totalShareholderEquity"]
          current_assets = bs_data["annualReports"][0]["totalCurrentAssets"]
          current_liabilities = bs_data["annualReports"][0]["totalCurrentLiabilities"]
          inventory = bs_data["annualReports"][0]["inventory"]

          #Some companies do not have any inventory, in such cases inventory is zero
          if inventory == "None":
            inventory = 0
          else:
            inventory = inventory = bs_data["annualReports"][0]["inventory"]

          #Converting fectched information into integers for calculations
          common_stock = int(common_stock)
          equity = int(equity)
          current_assets = int(current_assets)
          current_liabilities = int(current_liabilities)
          inventory = int(inventory)

        #Getting income statement information and coverting from string to int
        if alpha_vantage_income_statement:
          is_data = json.loads(alpha_vantage_income_statement.text)
          net_income = is_data["annualReports"][0]["netIncome"]
          net_income = int(net_income)

        #Getting beta information
        if beta_info:
          beta_data = json.loads(beta_info.text)
          beta = beta_data["data"]
          beta = float(beta)
          beta = round(beta, 2)


          #Converting close price into float
          for item in close_list:
            int_list.append(float(item))

          #Formating dates
          from datetime import datetime
          formatted_dates = [datetime.strptime(dt, "%Y-%m-%d").strftime("%d-%m") for dt in date_list]

          #getting length from dates
          length = len(formatted_dates)

          #Conevrting dates into numbers so they can fit into graph
          for i in range(length):
            number_list.append(i + 1)

          #Stock information Caclculations
          eps = net_income / common_stock
          pe = int_list[0] / eps
          roe = (net_income / equity) * 100
          quick_ratio = (current_assets - inventory) / current_liabilities
          expected_return = risk_free + (beta * (expected_rate - risk_free))
          expected_return = expected_return * 100

          #Calculating the value of the stock one year at a time and appending to a list
          for i in range(1, n + 1):
            value_over_time = calculate_future_value(i, expected_return, pv, pmt)
            value_over_time_list.append(value_over_time)
            years_list.append(i)

          #Plotting stock information hypotheical grwoth
          plt.title(f"{stock} Hypothetical Growth Over {n} Years")
          plt.xlabel(f"over {n} Years")
          plt.ylabel("Investment Value")
          plt.plot(value_over_time_list, years_list)
          plt.show()

          #Reversing list to show chart more accurately
          int_list.reverse()

          #Current stock price information
          plt.title(f"{stock} Stock Price Over Time")
          plt.xlabel(f"Past {length} Days")
          plt.ylabel(f"{stock} Stock Price")
          plt.plot(number_list, int_list)
          plt.show()

          #showing user calculated informatino
          print("Important Ratios for Investors")
          print(f"Earnings Per Share (EPS): {round(eps, 2)}" )
          print(f"Price Earnings (PE): {round(pe, 2)}")
          print(f"Return on Equity (ROE): {round(roe,2)}%")
          print(f"Quick Ratio: {round(quick_ratio,2)}")
          print(f"Expected Return (CAPM): {round(expected_return,2)}%")

          #Clearing List for next iteration
          close_list.clear()
          date_list.clear()
          int_list.clear()
          number_list.clear()
          value_over_time_list.clear()
          years_list.clear()

          #Asking user if they want to repeat
          repeat = input("Would you like to repeat (Yes or No)? ").lower()
          if repeat == "no":
            repeat = False
          else:
            repeat = True

      #Option 2 for user
      elif user_input == "2":
        #Getting required infomormation for TMV
        age = int(input("How old are you?\n"))
        stocks_profolio = (100 - age) / 100
        amount = int(input("How much money would you like to invest right now?\n"))
        n = int(input("What is your time horizon?"))
        pmt = int(input("How much money would you like to contribute yearly? "))

        #Calculating recommened profolio
        pv = stocks_profolio * amount
        random_numbers = random.sample(range(1, 7000), 10)
        bonds = age / 100
        amount_divided_evenly = amount / 10
        weight = 1/10

        #Opening CSV file for selection of random stocks
        with open("listing_status.csv", "r") as file:
          reader = csv.reader(file)
          for line in reader:
            asset_type = line[3]
            #There are other assets in CSV, so have to filter other assets out
            if asset_type == "Stock":
              symbol = line[0]
              name = line[1]
              name_list.append(name)
              symbol_list.append(symbol)


        #Displaying reccomendation for user
        print(f"Your profolio should be {stocks_profolio * amount}$ in  stocks and {bonds * amount}$ in bonds \n")
        print("Here are 10 randomly selected stocks options\n")

        #Selecting random stocks from CSV file
        for numbers in random_numbers:
          print(f"Name: {name_list[numbers]}, Symbol: {symbol_list[numbers]}")
          selected_stocks.append(symbol_list[numbers])

        print()


        #Getting Beta information for expected return
        for stock in selected_stocks:
          beta_info = requests.get(f"https://api.newtonanalytics.com/stock-beta/?ticker={stock}&index=^GSPC&interval=1mo​&observations=100")

          #For some stocks beta information is not available or there is not enough data to calculate it for the API
          #In either case beta will be set to 1 by default
          if beta_info:
            beta_data = json.loads(beta_info.text)
            beta = beta_data["data"]
            if beta == "Failure status when retrieving prices. ":
              beta = 1
            elif beta == "Statistics error: not enough observations to make calculation.":
              beta = 1
            else:
              beta = float(beta)
              beta = round(beta, 2)

          #Calculations of expected return
          expected_return = risk_free + (beta * (expected_rate - risk_free))
          expected_return = expected_return * 100
          expacted_return_list.append(expected_return / 100)

          #Displays Calcualted information about each stock to the user
          print(f"Expected Return for {stock}: {round(expected_return,2)}% \n")

        #Appending calculated expected return for each stock into a list to calculate profolio expected return
        for rate in expacted_return_list:
          profolio_expected = weight * rate
          profolio_expected_list.append(profolio_expected)

        #Calculating profolio expected return
        rec_rate = sum(profolio_expected_list) * reces
        norm_rate = sum(profolio_expected_list) * normal
        boom_rate = sum(profolio_expected_list) * boom
        expected_profolio_return = (rec_rate + norm_rate + boom_rate) * 100
        expected_profolio_return = round(expected_profolio_return, 2)
        print(f"Expected Profolio Return {expected_profolio_return}")

        #Calculating value of profolio one year at a time
        for i in range(1, n + 1):
          value_over_time = calculate_future_value(i, expected_profolio_return, pv, pmt)
          profolio_value_over_time_list.append(value_over_time)
          years_profolio_list.append(i)

        #Plotting of the values and showing user
        plt.title(f"Profolio Value Over Time")
        plt.xlabel("Years")
        plt.ylabel("Profolio Value")
        plt.plot(years_profolio_list, profolio_value_over_time_list)
        plt.show()

        repeat = input("Would you like to repeat (Yes or No)? ").lower()
        if repeat == "no":
          repeat = False
        elif repeat == "yes":
          repeat = True
        else:
          print("Invalid Input")
        plt.show()

        #Clearing list for next iteration
        random_num.clear()
        name_list.clear()
        symbol_list.clear()
        selected_stocks.clear()
        expacted_return_list.clear()
        profolio_expected_list.clear()
        profolio_value_over_time_list.clear()
        years_profolio_list.clear()

      elif user_input == "3":
        stock = yf.Ticker(input("Enter a stock (Stock Symbol)"))
        stock = stock.history(period = "max")

        #Deleting Dividends and Stock split from data frame
        del stock["Dividends"]
        del stock["Stock Splits"]

        #Plotting Data
        stock.index
        stock.plot.line(y="Close", use_index = True)

        #Set up target to predict using machine learning
        #Shifting data back one day
        stock["Tomorrow"] = stock["Close"].shift(-1)

        #Target: 1 when price went up 0 when price went down
        stock["Target"] = (stock["Tomorrow"] > stock["Close"]).astype(int)

        #Only dates after 1990
        stock = stock.loc["1990-01-01":].copy()
        stock

        #Importing Random forrest
        from sklearn.ensemble import RandomForestClassifier

        #Initializing model
        model = RandomForestClassifier(n_estimators = 200, min_samples_split = 50, random_state = 1)
        train = stock.iloc[:-100]
        test = stock.iloc[-100:]
        predictors = ["Close", "Volume", "Open", "High", "Low"]
        model.fit(train[predictors], train["Target"])

        from sklearn.metrics import precision_score
        #How accurate is the model
        preds = model.predict(test[predictors])
        preds = pd.Series(preds, index = test.index)
        precision_score(test["Target"], preds)
        combined = pd.concat([test["Target"], preds], axis = 1)
        combined.plot()

        predictions = backtest(stock, model, predictors)
        predictions["Predictions"].value_counts()
        #precision_score(predictions["Target"], predictions["Predictions"])
        predictions["Target"].value_counts() / predictions.shape[0]

        #Rolling averages
        horizons = [2, 5, 60, 1000]

        new_predictors = []
        for horizon in horizons:
          rolling_averages = stock.rolling(horizon).mean()
          ratio_column = f"Close_Ratio_{horizon}"
          stock[ratio_column] = stock["Close"] / rolling_averages["Close"]
          trend_column = f"Trend_{horizon}"
          stock[trend_column] = stock.shift(1).rolling(horizon).sum()["Target"]
          new_predictors += [ratio_column, trend_column]
        stock = stock.dropna()
        stock

        predictions = backtest(stock, model, new_predictors)
        print(predictions["Predictions"].value_counts())
        print(precision_score(predictions["Target"], predictions["Predictions"]))

      else:
        print("Invalid Input")
        repeat = input("Would you like to repeat (Yes or No)? ").lower()
        if repeat == "no":
          repeat = False
        else:
          repeat = True

  except:
    print("An Unkown error has occured")